In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/openwebtext/openwebtext.tar.xz


* <font color="red">將`/kaggle/input/openwebtext/openwebtext.tar.xz`解壓到**Output**，成為2萬多個(較小)xz檔</font>

In [1]:
!tar -xf /kaggle/input/openwebtext/openwebtext.tar.xz

In [ ]:
# !ls /kaggle/working/openwebtext ## 列出目錄中所有檔名 

* <font color="red">用以建立一個`list`，包含目錄下所有`xz`檔的名稱</font>

In [4]:
def xz_files_in_dir(directory):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith(".xz") and os.path.isfile(os.path.join(directory, filename)):
           files.append(filename)
    return files 

In [10]:
# xz_files_in_dir("/kaggle/working/openwebtext")[:5] ## 列出目錄中前5個檔案名 

['urlsf_subset15-383_data.xz',
 'urlsf_subset20-208_data.xz',
 'urlsf_subset19-286_data.xz',
 'urlsf_subset18-554_data.xz',
 'urlsf_subset05-460_data.xz']

In [8]:
from tqdm import tqdm
import lzma

In [14]:
!ls /kaggle/working/

openwebtext  output1.txt  vocab.txt


In [21]:
len(text)

2174764

* <font color="red">原始1版--將**Output**中，`openwebtext`目錄下的xz檔依次解壓，並讀取</font>
> * <font color="red">`split_files`指定了，所有的xz檔解壓後，轉存成多少個txt檔!</font>
> * <font color="red">`vocab_file`儲存所有(在xz檔解壓後)出現過的character(不重複)</font>
> * <font color="red">**Output**只有`19.5GB`,xz檔案就佔用了`12GB`，解壓後轉存成txt檔，只能處理`17%`</font>

In [13]:
folder_path = "/kaggle/working/openwebtext" ## xz檔案 所在的目錄
output_file = "/kaggle/working/output{}.txt" ## xz解壓後，所轉換成的txt檔案
vocab_file = "/kaggle/working/vocab.txt"
split_files = int(input("How many files would you like to split this into?")) ## 輸入:1

files = xz_files_in_dir(folder_path)
total_files = len(files)              ## 20610 個 檔案

max_count = total_files // split_files if split_files != 0 else total_files ## 每一個txt所包含的xz檔案數目

vocab = set() ## 建立一個 set，將用來存 char

for i in range(split_files):
    with open(output_file.format(i),"w", encoding="utf-8") as outfile:
        for count, filename in enumerate(tqdm(files[:max_count], total=max_count)): ## total 用已顯示 bar state
            if count >= max_count:
                 break
            file_path = os.path.join(folder_path, filename)
            with lzma.open(file_path, "rt", encoding="utf-8") as infile:
                 text = infile.read()
                 outfile.write(text)
                 characters = set(text)
                 vocab.update(characters)
        files = files[max_count:]

with open(vocab_file, "w", encoding="utf-8") as vfile:
     for char in vocab:
         vfile.write(char + '\n')

 17%|█▋        | 3601/20610 [06:46<32:00,  8.86it/s]  


OSError: [Errno 28] No space left on device

* <font color="red">原始2版+一個修改--將**Output**中，`openwebtext`目錄下的xz檔依次解壓，並讀取</font>
> * <font color="red">`90%`的xz檔解壓後，轉存成`output_train.txt`</font>
> * <font color="red">`10%`的xz檔解壓後，轉存成`output_val.txt`</font>
> * <font color="red">一個(自訂)修改--`os.remove(file_path)`,當一個xz檔被讀取後，即被刪除。此法的確可以讀取更多xz檔，但空間依然不夠。在`Process the training files`的過程中，只處理了27%，就產生`空間不足`的錯誤</font> 

In [7]:
folder_path = "/kaggle/working/openwebtext"
output_file_train = "/kaggle/working/output_train.txt"
output_file_val = "/kaggle/working/output_val.txt"
vocab_file = "/kaggle/working/vocab.txt"

files = xz_files_in_dir(folder_path)
total_files = len(files)

# Calculate the split indices
split_index = int(total_files * 0.9) # 90% for training
files_train = files[:split_index]
files_val = files[split_index:]

# Process the files for training and validation separately
vocab = set()

# Process the training files
with open(output_file_train, "w", encoding="utf-8") as outfile:
    for filename in tqdm(files_train, total=len(files_train)):
        file_path = os.path.join(folder_path, filename)
        with lzma.open(file_path, "rt", encoding="utf-8") as infile:
            text = infile.read()
            outfile.write(text)
            characters = set(text)
            vocab.update(characters)
        os.remove(file_path) ## 自行測試
        
# Process the validation files
with open(output_file_val, "w", encoding="utf-8") as outfile:
    for filename in tqdm(files_train, total=len(files_val)):
        file_path = os.path.join(folder_path, filename)
        with lzma.open(file_path, "rt", encoding="utf-8") as infile:
            text = infile.read()
            outfile.write(text)
            characters = set(text)
            vocab.update(characters)

# Write the vocabulary to vocab.txt
with open(vocab_file, "w", encoding="utf-8") as vfile:
     for char in vocab:
         vfile.write(char + '\n')

 27%|██▋       | 4997/18549 [08:55<24:12,  9.33it/s]  


OSError: [Errno 28] No space left on device

* <font color="red">解壓/讀取一個xz檔，儲存成一個txt檔</font>

In [11]:
folder_path = "/kaggle/working/openwebtext" ## xz檔案 所在的目錄
output_file = "/kaggle/working/output{}.txt" ## xz解壓後，所轉換成的txt檔案
vocab_file = "/kaggle/working/vocab.txt"


files = xz_files_in_dir(folder_path)
total_files = len(files)              ## 20610 個 檔案

vocab = set() ## 建立一個 set，將用來存 char

out_n = 1      ## 存成一個txt檔
max_count = 1  ## 解壓/讀取一個xz檔

with open(output_file.format(out_n),"w", encoding="utf-8") as outfile:
    for count, filename in enumerate(tqdm(files[:max_count], total=max_count)): ## total 用已顯示 bar state
        if count >= max_count:
             break
        file_path = os.path.join(folder_path, filename)
        with lzma.open(file_path, "rt", encoding="utf-8") as infile:
             text = infile.read()
             outfile.write(text)
             characters = set(text)
             vocab.update(characters)
#         os.remove(file_path) ## 自行測試

with open(vocab_file, "w", encoding="utf-8") as vfile:
     for char in vocab:
         vfile.write(char + '\n')

100%|██████████| 1/1 [00:00<00:00,  8.18it/s]


In [ ]:
# text.replace('\x00','') ## 將text中`\x00`給取代掉